### Code to run web Interface and Arduino ###

#### 1. Establish connection with Arduino
#### 2. Run interface using Flask

In [1]:
#Function to find current Arduino Port:
def find_port(ports):
    
    for port in ports:
        try:
            ser = serial.Serial(port,9600,timeout = 1)
            print("Device Found at: ",port)
            device_port = port
            break
        except:
            next
    return device_port 


#Function to Bootup Arduino:
def boot_arduino(arduino):
    time.sleep(1)
    #send Startup code to the Arduino
    arduino.write(b'Startup#')
    print("Message sent")
    flag = 0
    seq = []
    count = 1
    #waiting for bootup to complete
    while flag == 0:
        for c in arduino.read():
            seq.append(chr(c)) #convert from ANSII
            joined_seq = ''.join(str(v) for v in seq) #Make a string from array

            if chr(c) == '\n':
                print("Line " + ': ' + joined_seq)
                seq = []
                flag = 1
                break


In [ ]:
#Import require Flask library
from flask import Flask,redirect,url_for, render_template, request, session, Response
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import desc
from datetime import datetime
from random import randrange

## IMPORT ARDUINO CODE
import serial
from time import sleep
import time

#custom module
from camera import Camera

#SETUP ARDUINO
#ports = ['COM3','COM4','COM5','COM6','COM7','COM8','COM10'] #List of available ports
#port = find_port(ports)  #call find_port function

arduino = serial.Serial('/dev/cu.usbmodem14601',9600,timeout= 2)
boot_arduino(arduino) #call boot_arduino function

#creating teflask app
app1 = Flask(__name__)
app1.secret_key = "HELLO"
#configuring SQLALCHEMY_DATABASE
app1.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.sqlite3'
app1.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False

label1 = 0
people_in = 0

#creating database for app
db = SQLAlchemy(app1)

#configuring visitors table in the database 
class visitors(db.Model):
    _id = db.Column("id", db.Integer, primary_key=True) #unique id number
    operation = db.Column(db.String(100)) #opration type
    value = db.Column(db.Integer)  #visitor count value
    total_count = db.Column(db.Integer)  #Total count of visitor
    timestamp = db.Column(db.DateTime)  #timestamp on each entry
    
    def __init__(self, _id, operation, value, total_count, timestamp):
        self._id = _id
        self.operation = operation
        self.value = value
        self.total_count = total_count
        self.timestamp = timestamp
    
#Function to define override mode
def over(value):
    session["id"] += 100
    #multiple override
    if value == "Multiple":
        session["override"] = 2
        rand_val = people_in
        session["visitor"] += rand_val
        #add visitor
        visit = visitors(session["id"],"Manual Override", rand_val, session["visitor"], datetime.now())
    #single override
    elif value == "Single":
        session["override"] = 1
        session["visitor"] += 1
        #add visitor
        visit = visitors(session["id"],"Single Override", 1, session["visitor"], datetime.now())
    #adding data in database
    db.session.add(visit)
    #saving the transaction 
    db.session.commit()

#Flask module for home page
@app1.route("/", methods=["POST","GET"])
def home():
    
    
    global label1
    
    #perform actions based on type of request
    if request.method == "POST":
        val = request.form["do"]
        if val == "Cancel":
              session["override"] = 0
        else:
            over(val)
        return redirect(url_for("home"))
    else:
        if label1 == 0:
            session["visitor"] = 0
            session["override"] = 0
            session["id"] = 0
            label1 = 1
        return render_template("home.html", values= session["visitor"])
        
#function to get webcam input
def gen(camera,session,db):
    
    seq = [] #list to receive arduino message
    reset_time = 0 #variable to keep time
    time_limit = 7 #maximum time limit for entering the gate
    global people_in #active people count
    change = 0 #number of people walk-in or out
    
    prediction_history = [-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1] #array for smoothening predictions
    
    classifier_dict={-1:'no_person',0:'without_mask',1:'fabric',2:'n95',3:'surgical'}
    
    while True:

        if(arduino.inWaiting()>0):
            for c in arduino.read(): #read arduino
                seq.append(chr(c)) #convert from ANSII
                joined_seq = ''.join(str(v) for v in seq) #Make a string from array
                
                if chr(c) == '\n':
                    print("Line " + ': ' + joined_seq)
                    seq = []
                    try:
                        change = int(joined_seq)
                        people_in += change
                        print("People IN: ",people_in)
                        
                    except:
                        
                        people_in = 0
                        
                    break

        frame,mask_flag,prediction_history = camera.get_frame(prediction_history)
        print(classifier_dict[mask_flag] , " \n Prediction Array: " ,prediction_history)
        
        #yield camera frame
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
        
        #send message to the arduino to open the gate in mask is detected 
        if ( (mask_flag > 0) and ( (change > 0) or ( (time.time() - reset_time) > time_limit  ) ) ):
            
            prediction_history = [-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1] #to reset array after opening the gate
            arduino.write(b'Gates#') #command arduino to open gates
            print("Sent to Arduino")
            reset_time = time.time()
            change = 0
    
#Flask module for video_feed 
@app1.route("/video_feed")
def video_feed():
    
    return Response(gen(Camera(),session,db),
                    mimetype='multipart/x-mixed-replace; boundary=frame')
    
    
    

#Flask module for visitor page 
@app1.route("/visitor", methods=["POST","GET"])
def visitor():
    
    global people_in
    
    if(people_in != 0): #update database if people enter or exit
        
        session["id"] += 5
        session["visitor"] += people_in
        visit = visitors(session["id"],"Visitor_Page", people_in, session["visitor"], datetime.now())
        db.session.add(visit)
        db.session.commit()
        people_in = 0
        
    
    if request.method == "POST":

        val = request.form["do"]
        vis(val)
        return redirect(url_for("visitor"))
    else:
        return render_template("visitor.html", values = visitors.query.order_by(desc(visitors.timestamp)).all())

        
    
if __name__ == "__main__":
    label1 = 0
    people_in = 0
    db.drop_all()
    db.create_all()
    app1.run()


Message sent
Line : Device Booted Successfully

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Nov/2020 22:44:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Nov/2020 22:44:30] "GET /video_feed HTTP/1.1" 200 -


-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0]
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, 0]
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, 0, 0, 0]
Line : -2

People IN:  -2
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, 0, 0, 0, 2]
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, 0, 0, 0, 2, 0, 0]
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2]
0
without_mask  
 Prediction Array:  [0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0]
0
without_mask  
 Prediction Array:  [0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0]
0
without_mask  
 Prediction Array:  [0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0]
0
without_mask  
 Prediction Array:  [0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2]
0
without_mask  
 Prediction Array:  [0, 0, 0, 0, 0, 2, 0, 0, 2

3
surgical  
 Prediction Array:  [3, 3, 3, 3, 3, -1, -1, -1, -1, -1, -1]
3
surgical  
 Prediction Array:  [3, 3, 3, 3, 3, -1, -1, -1, -1, -1, -1, -1]
Sent to Arduino
Line : 0

People IN:  -2
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
Line : -4

People IN:  -6
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3]
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, 3]
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, 3, 3]
-1
no_person  
 Predi

-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, 2, -1, 3, -1, -1, 3, 3, 3, 3, 3, 3]
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, 2, -1, 3, -1, -1, 3, 3, 3, 3, 3, 3, 3]
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, 2, -1, 3, -1, -1, 3, 3, 3, 3, 3, 3, 3, 3]
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, 2, -1, 3, -1, -1, 3, 3, 3, 3, 3, 3, 3, 3, 3]
3
surgical  
 Prediction Array:  [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
Sent to Arduino
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3]
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, 3]
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, 3, 3]


127.0.0.1 - - [05/Nov/2020 22:45:44] "GET /visitor HTTP/1.1" 200 -


-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, 3, 3, -1]


Error on request:
Traceback (most recent call last):
  File "/Users/apple/opt/anaconda3/lib/python3.7/site-packages/werkzeug/serving.py", line 323, in run_wsgi
    execute(self.server.app)
  File "/Users/apple/opt/anaconda3/lib/python3.7/site-packages/werkzeug/serving.py", line 315, in execute
    write(data)
  File "/Users/apple/opt/anaconda3/lib/python3.7/site-packages/werkzeug/serving.py", line 296, in write
    self.wfile.write(data)
  File "/Users/apple/opt/anaconda3/lib/python3.7/socketserver.py", line 799, in write
    self._sock.sendall(b)
OSError: [Errno 41] Protocol wrong type for socket
127.0.0.1 - - [05/Nov/2020 22:45:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Nov/2020 22:45:51] "GET /video_feed HTTP/1.1" 200 -


-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3]
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, 3]
Line : 0

People IN:  0
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, 3, 3]
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, 3, 3, 3]
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, 3, 3, 3, 3]
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, 3, 3, 3, 3, 2]
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, 3, 3, 3, 3, 2, 3]
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, 3, 3, 3, 3, 2, 3, 3]
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, 3, 3, 3, 3, 2, 3, 3, 3]
-1
no_person  
 Prediction Array:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3]
3
surgical 